In [86]:
from transformers import AutoProcessor, AutoModelForVision2Seq
import torch
model = AutoModelForVision2Seq.from_pretrained(
    "HuggingFaceTB/SmolVLM-256M-Instruct",
    torch_dtype=torch.bfloat16,
    # _attn_implementation="flash_attention_2" if DEVICE == "cuda" else "eager",
)

In [87]:
new_tokens = ['<detect>',*[f'<loc{v:04d}>' for v in range(1024)]]
processor = AutoProcessor.from_pretrained("HuggingFaceTB/SmolVLM-256M-Instruct", additional_special_tokens=new_tokens)
len(new_tokens), new_tokens[:5]

(1025, ['<detect>', '<loc0000>', '<loc0001>', '<loc0002>', '<loc0003>'])

In [88]:

model.resize_token_embeddings(len(processor.tokenizer))

Embedding(50305, 576, padding_idx=2)

In [89]:
from peft import LoraConfig, get_peft_model

In [90]:
targets = ['q_proj','k_proj','v_proj','out_proj','gate_proj','up_proj','down_proj']

In [91]:
config = LoraConfig(
    r=64,
    lora_alpha=16,
    target_modules=targets,
    lora_dropout=0.1,
    bias="none",
    modules_to_save=['embed_tokens','lm_head'],
    inference_mode=False,
    task_type='CAUSAL_LM'
)
model = get_peft_model(model, config)
model.get_output_embeddings().modules_to_save.default.weight = model.get_input_embeddings().modules_to_save.default.weight
model.print_trainable_parameters()

trainable params: 51,020,352 || all params: 308,686,080 || trainable%: 16.5282
